In [1]:
# Import các thư viện cần thiết
import numpy as np 
import torch 
from torch.nn import Parameter 
from torch.nn.modules.module import  Module 
import torch.nn.functional as F
import math 
from torch import nn 

In [2]:
# Xây dựng BaseModel 
class PruningModel(Module):
	def prune_by_std(self, s=0.25):
		"""
			Cắt tỉa trọng số sử dụng độ lệch chuẩn (std - 
			standard deviation) được gọi là ngưỡng (threshold)
		"""
		# Lưu ý rằng thuật ngữ `module`` ở đây được hiểu là các 
		# `lớp (layer)`, ví dụ: fc1, fc2, fc3
		for name, module in self.named_modules():
			if name in ['fc1', 'fc2', 'fc3']:
				threshold = np.std(module.weight.data.cpu().numpy()) * s
				print(f">>> Cắt tỉa với ngưỡng: {threshold} cho layer {name}")
				module.prune(threshold)

In [3]:
# Xây dựng module cắt tỉa 
class MaskedLinear(Module):
	def __init__(self, in_features, out_features, bias=True):
		super(MaskedLinear, self).__init__()
		self.in_features = in_features
		self.out_features = out_features
		self.weight = Parameter(torch.Tensor(out_features, in_features))
		# Khởi tạo bộ lọc (mask-mặt nạ) cho ta quyết định 
		# weight nào được tính toán, weight nào không
		self.mask = Parameter(
			torch.ones([out_features, in_features]), 
			requires_grad=False
		)
		if bias: 
			self.bias = Parameter(torch.Tensor(out_features))
		else:
			self.register_parameter('bias', None)
		self.reset_parameters() 

	def reset_parameters(self):
		stdv = 1. / math.sqrt(self.weight.size(1))
		# Phân phối đều trọng số mô hình trong khoảng [-stdv, stdv]
		self.weight.data.uniform_(-stdv, stdv)
		if self.bias is not None:
			self.bias.data.uniform_(-stdv, stdv)

	def forward(self, input):
		# Nhân weight với bộ lọc trước. Điều này giúp loại 
		# bỏ đi các weight không cần thiết sau khi đã cắt tỉa
		return F.linear(input, self.weight * self.mask, self.bias)

	def __repr__(self):
		return self.__class__.__name__ + '(' \
			+ 'in_features=' + str(self.in_features) \
			+ ', out_features=' + str(self.out_features) \
			+ ', bias=' + str(self.bias is not None) + ')'

	def prune(self, threshold):
		"""
			Hàm tuỳ chỉnh cắt tỉa (prune) với bộ lọc (mask). Tại mỗi 
			lần cắt tỉa, tính toán các trong số nào có weight nhỏ 
			hơn ngưỡng quy định, cập nhật lại bộ lọc (mask) và weight 
			tại các vị trí đó về giá trị 0.
		"""
		weight_device = self.weight.device
		mask_device = self.mask.device 
		# Đưa tensor từ GPU về CPU và chuyển tensor về mảng numpy
		tensor = self.weight.data.cpu().numpy()
		mask = self.mask.data.cpu().numpy()
		# Sau khi cắt tỉa những weight nào không cần nữa thì sẽ thành số 0
		new_mask = np.where(abs(tensor) < threshold, 0, mask)
		# Apply trọng số và bộ lọc (mask-mặt nạ) mới
		self.weight.data = torch.from_numpy(tensor * new_mask).to(weight_device)
		self.mask.data = torch.from_numpy(new_mask).to(mask_device)

In [4]:
# Cài đặt mạng FullyConnected, kết nối tất cả module (layer-lớp) lại với nhau
class LeNet(PruningModel):
	def __init__(self, mask=False):
		super(LeNet, self).__init__()
		linear = MaskedLinear if mask else nn.Linear 
		self.fc1 = linear(784, 300)
		self.fc2 = linear(300, 100)
		self.fc3 = linear(100, 10)

	def forward(self, x):
		x = x.view(-1, 784)
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		x = F.log_softmax(self.fc3(x), dim=1)
		return x 

In [5]:
# Cài đặt hyperparameter 
# Define some const

BATCH_SIZE = 128
EPOCHS = 100
LEARNING_RATE = 0.001
USE_CUDA = True
SEED = 42
LOG_AFTER = 10 # How many batches to wait before logging training status
LOG_FILE = 'log_prunting.txt'
SENSITIVITY = 2 # Sensitivity value that is multiplied to layer's std in order to get threshold value

# Control Seed
torch.manual_seed(SEED)

# Select Device
use_cuda = USE_CUDA and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else 'cpu')

In [6]:
# Cài đặt DataLoader 

# Tạo tập dữ liệu MNIST 
from torchvision import datasets, transforms 

# Train loader 
kwargs = {"num_workers": 5, "pin_memory": True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
	datasets.MNIST(
		'data', train=True, download=True, 
		transform=transforms.Compose([
			transforms.ToTensor(), 
			transforms.Normalize((0.1307, ), (0.3081, ))
		])), 
	batch_size=BATCH_SIZE, shuffle=True, **kwargs
)
# Test loader
test_loader = torch.utils.data.DataLoader(
	datasets.MNIST(
		'data', train=False, transform=transforms.Compose([
		transforms.ToTensor(),
		transforms.Normalize((0.1307,), (0.3081,))
	])),
	batch_size=BATCH_SIZE, shuffle=False, **kwargs)

In [7]:
model = LeNet(mask=True).to(device)
model

LeNet(
  (fc1): MaskedLinear(in_features=784, out_features=300, bias=True)
  (fc2): MaskedLinear(in_features=300, out_features=100, bias=True)
  (fc3): MaskedLinear(in_features=100, out_features=10, bias=True)
)

In [8]:
import torch.optim as optim

# Định nghĩa AdamOptimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=0.0001)
initial_optimizer_state_dict = optimizer.state_dict()

In [9]:
from tqdm import tqdm 

def train(model):
	model.train()
	for epoch in range(EPOCHS):
		pbar = tqdm(enumerate(train_loader), total=len(train_loader))
		for batch_idx, (data, target) in pbar:
			data, target = data.to(device), target.to(device)
			optimizer.zero_grad()
			output = model(data)
			loss = F.nll_loss(output, target)
			loss.backward()
			
			# Đặt tất cả các gradient tương ứng với các kết nối đã bị cắt tỉa về 0
			# Hàm này sẽ không chạy trong lần đầu training mà sẽ chạy sau khi mạng 
			# đã được cắt tỉa và cần fine-tuning lại. Giúp optimizer chỉ tối ưu vào 
			# các trọng số chưa được cắt tỉa (quan trọng)
			for name, p in model.named_parameters():
				if 'mask' in name:
					continue
				tensor = p.data.cpu().numpy()
				grad_tensor = p.grad.data.cpu().numpy()
				grad_tensor = np.where(tensor==0, 0, grad_tensor)
				p.grad.data = torch.from_numpy(grad_tensor).to(device)
			
			optimizer.step()
			if batch_idx % LOG_AFTER == 0:
				done = batch_idx * len(data)
				percentage = 100. * batch_idx / len(train_loader)
				pbar.set_description(f'Train Epoch: {epoch} [{done:5}/{len(train_loader.dataset)} ({percentage:3.0f}%)]-----Loss: {loss.item():.6f}')
	return model

In [10]:
model = train(model)

Train Epoch: 0 [58880/60000 ( 98%)]-----Loss: 0.196225: 100%|██████████| 469/469 [00:01<00:00, 342.10it/s]
Train Epoch: 1 [58880/60000 ( 98%)]-----Loss: 0.175027: 100%|██████████| 469/469 [00:01<00:00, 365.67it/s]
Train Epoch: 2 [58880/60000 ( 98%)]-----Loss: 0.100646: 100%|██████████| 469/469 [00:01<00:00, 368.16it/s]
Train Epoch: 3 [58880/60000 ( 98%)]-----Loss: 0.048760: 100%|██████████| 469/469 [00:01<00:00, 384.83it/s]
Train Epoch: 4 [58880/60000 ( 98%)]-----Loss: 0.008782: 100%|██████████| 469/469 [00:01<00:00, 399.82it/s]
Train Epoch: 5 [58880/60000 ( 98%)]-----Loss: 0.109470: 100%|██████████| 469/469 [00:01<00:00, 330.62it/s]
Train Epoch: 6 [58880/60000 ( 98%)]-----Loss: 0.049825: 100%|██████████| 469/469 [00:01<00:00, 323.31it/s]
Train Epoch: 7 [58880/60000 ( 98%)]-----Loss: 0.065000: 100%|██████████| 469/469 [00:01<00:00, 317.03it/s]
Train Epoch: 8 [58880/60000 ( 98%)]-----Loss: 0.004740: 100%|██████████| 469/469 [00:01<00:00, 315.19it/s]
Train Epoch: 9 [58880/60000 ( 98%)]--

In [11]:
from time import time 

def test(model):
	start_time = time()
	model.eval()
	test_loss = 0
	correct = 0
	with torch.no_grad():
		for data, target in test_loader:
			data, target = data.to(device), target.to(device)
			output = model(data)
			test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
			pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
			correct += pred.eq(target.data.view_as(pred)).sum().item()

		test_loss /= len(test_loader.dataset)
		accuracy = 100. * correct / len(test_loader.dataset)
		print(f'>>> Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%). Total time = {time() - start_time}')
	return accuracy

In [12]:
accuracy = test(model)

>>> Test set: Average loss: 0.0809, Accuracy: 9814/10000 (98.14%). Total time = 0.16672825813293457


In [13]:
# Lưu lại giá trị log vào file để theo dõi
def save_log(filename, content):
	with open(filename, 'a') as f:
		content += "\n"
		f.write(content)

In [14]:
save_log(LOG_FILE, f"initial_accuracy {accuracy}")
torch.save(model, f"save_models/initial_model.ptmodel")

In [15]:
# Tính số lượng non-zeros parameters
def print_nonzeros(model):
	"""Hiển thị số lượng trọng số non-zeros của model (mô hình)"""
	nonzero = total = 0
	for name, p in model.named_parameters():
		if 'mask' in name:
			continue
		tensor = p.data.cpu().numpy()
		nz_count = np.count_nonzero(tensor)
		total_params = np.prod(tensor.shape)
		nonzero += nz_count
		total += total_params
		print(f'{name:20} | nonzeros = {nz_count:7} / {total_params:7} ({100 * nz_count / total_params:6.2f}%) | total_pruned = {total_params - nz_count :7} | shape = {tensor.shape}')
	print(f'alive: {nonzero}, pruned : {total - nonzero}, total: {total}, Compression rate : {total/nonzero:10.2f}x  ({100 * (total-nonzero) / total:6.2f}% pruned)')

In [16]:
print_nonzeros(model)
# Có thể thấy rằng khi chưa được cắt tỉa thì mạng này
# có toàn bộ các weight là khác 0. Tức là hiện tại chưa 
# có weight nào được cắt tỉa (0.0% pruned).

fc1.weight           | nonzeros =  235200 /  235200 (100.00%) | total_pruned =       0 | shape = (300, 784)
fc1.bias             | nonzeros =     300 /     300 (100.00%) | total_pruned =       0 | shape = (300,)
fc2.weight           | nonzeros =   30000 /   30000 (100.00%) | total_pruned =       0 | shape = (100, 300)
fc2.bias             | nonzeros =     100 /     100 (100.00%) | total_pruned =       0 | shape = (100,)
fc3.weight           | nonzeros =    1000 /    1000 (100.00%) | total_pruned =       0 | shape = (10, 100)
fc3.bias             | nonzeros =      10 /      10 (100.00%) | total_pruned =       0 | shape = (10,)
alive: 266610, pruned : 0, total: 266610, Compression rate :       1.00x  (  0.00% pruned)


In [17]:
# Tiến hành cắt tỉa
model.prune_by_std(SENSITIVITY)

>>> Cắt tỉa với ngưỡng: 0.07601112872362137 cho layer fc1
>>> Cắt tỉa với ngưỡng: 0.11727409809827805 cho layer fc2
>>> Cắt tỉa với ngưỡng: 0.37368443608283997 cho layer fc3


In [18]:
# Chạy test lại độ chính xác sau khi cắt tỉa
accuracy = test(model)

>>> Test set: Average loss: 0.9760, Accuracy: 6674/10000 (66.74%). Total time = 0.15550613403320312


In [19]:
# Lưu kết quả vào log file và kiểm tra lại số lượng tham số của mạng
save_log(LOG_FILE, f"accuracy_after_pruning {accuracy}")
print_nonzeros(model)
# Nhận xét: Mô hình giảm đi độ chính xác khá nhiều, từ 98.12% xuống còn 60.01%
# Trong khi số lượng tham số bị cắt tỉa là 94.11% (pruned) tương ứng tỷ lệ nén 
# khoảng 16.98x lần. Tiếp theo ta cần training lại PrunedNetwork (mạng sau khi 
# cắt tỉa)

fc1.weight           | nonzeros =   13450 /  235200 (  5.72%) | total_pruned =  221750 | shape = (300, 784)
fc1.bias             | nonzeros =     300 /     300 (100.00%) | total_pruned =       0 | shape = (300,)
fc2.weight           | nonzeros =    2103 /   30000 (  7.01%) | total_pruned =   27897 | shape = (100, 300)
fc2.bias             | nonzeros =     100 /     100 (100.00%) | total_pruned =       0 | shape = (100,)
fc3.weight           | nonzeros =      67 /    1000 (  6.70%) | total_pruned =     933 | shape = (10, 100)
fc3.bias             | nonzeros =      10 /      10 (100.00%) | total_pruned =       0 | shape = (10,)
alive: 16030, pruned : 250580, total: 266610, Compression rate :      16.63x  ( 93.99% pruned)


In [20]:
# Retraining PrunedNetwork (mạng sau khi cắt tỉa)
optimizer.load_state_dict(initial_optimizer_state_dict) # Reset the optimizer

model = train(model)

Train Epoch: 0 [58880/60000 ( 98%)]-----Loss: 0.019772: 100%|██████████| 469/469 [00:01<00:00, 315.47it/s]
Train Epoch: 1 [58880/60000 ( 98%)]-----Loss: 0.073803: 100%|██████████| 469/469 [00:01<00:00, 314.03it/s]
Train Epoch: 2 [58880/60000 ( 98%)]-----Loss: 0.007872: 100%|██████████| 469/469 [00:01<00:00, 317.05it/s]
Train Epoch: 3 [58880/60000 ( 98%)]-----Loss: 0.022281: 100%|██████████| 469/469 [00:01<00:00, 312.70it/s]
Train Epoch: 4 [58880/60000 ( 98%)]-----Loss: 0.018000: 100%|██████████| 469/469 [00:01<00:00, 316.28it/s]
Train Epoch: 5 [58880/60000 ( 98%)]-----Loss: 0.012123: 100%|██████████| 469/469 [00:01<00:00, 314.01it/s]
Train Epoch: 6 [58880/60000 ( 98%)]-----Loss: 0.014565: 100%|██████████| 469/469 [00:01<00:00, 309.40it/s]
Train Epoch: 7 [58880/60000 ( 98%)]-----Loss: 0.022879: 100%|██████████| 469/469 [00:01<00:00, 311.53it/s]
Train Epoch: 8 [58880/60000 ( 98%)]-----Loss: 0.003166: 100%|██████████| 469/469 [00:01<00:00, 311.71it/s]
Train Epoch: 9 [58880/60000 ( 98%)]--

In [21]:
# Chạy test lại độ chính xác của PrunedNetwork (mạng sau khi cắt tỉa)
accuracy = test(model)
# Nhận xét: Độ chính xác sau khi retraining PrunedNetwork (mạng sau khi 
# cắt tỉa, với tỷ lệ nén 16.63x) đã tăng từ 66.74% lên 98.08%. 

>>> Test set: Average loss: 0.0674, Accuracy: 9808/10000 (98.08%). Total time = 0.15483522415161133


In [22]:
save_log(LOG_FILE, f"accuracy_after_retraining {accuracy}")
torch.save(model, f"save_models/model_after_retraining.ptmodel")

In [23]:
# Tiếp theo để tiến hành tăng tỉ số nén chúng ta sẽ đến với phần lượng tử hóa và share weight 

# Lượng tử hoá và share weight
from sklearn.cluster import KMeans
from scipy.sparse import csc_matrix, csr_matrix

def apply_weight_sharing(model, bits=5):
	for module in model.children():
		devive = module.weight.device
		weight = module.weight.data.cpu().numpy()
		shape = weight.shape
		# Note:
		# 	Lưu trữ dưới dạng compressed sparse row (CSR) hoặc compressed sparse column 
		# 	(CSC) format là hai format để lưu trữ ma trận thưa nhằm tính toán được dễ 
		# 	dàng do tiết kiệm về bộ nhớ. Vì weight là là một ma trận rất thưa với 94% 
		# 	các trọng số là khác 0, nên cần phải có một cấu trúc phù hợp để lưu trữ và 
		# 	tính toán.
		mat = csr_matrix(weight) if shape[0] < shape[1] else csc_matrix(weight)
		# Sử dụng Kmean để phân cụm, được thực hiện như sau
		min_ = min(mat.data)
		max_ = max(mat.data)
		space = np.linspace(min_, max_, num=2**bits)
		kmeans = KMeans(
			n_clusters=len(space), 
			init=space.reshape(-1,1), n_init=1, 
			# precompute_distances=True, # Từ 0.24.0 trở đi, loại bỏ do đã được tối ưu hoá trực tiếp trong hàm KMeans
			# algorithm="full" # Từ 0.24.0 trở đi, tham số algorithm chỉ chấp nhận các giá trị 'elkan' và 'lloyd'
			algorithm='lloyd'
		)
		kmeans.fit(mat.data.reshape(-1,1))
		# Ở đây số lượng bits được sử dụng để lưu trữ các giá trị weight là 5. Nên ta 
		# có tối đa là 2^5=32 cụm của K-means. Sau khi thực hiện phân cụm xong thì ta 
		# tiến hành share lại centroid vào các vị trí weight bằng hàm.
		new_weight = kmeans.cluster_centers_[kmeans.labels_].reshape(-1)
		mat.data = new_weight
		module.weight.data = torch.from_numpy(mat.toarray()).to(devive)
	return model

apply_weight_sharing(model=model)

LeNet(
  (fc1): MaskedLinear(in_features=784, out_features=300, bias=True)
  (fc2): MaskedLinear(in_features=300, out_features=100, bias=True)
  (fc3): MaskedLinear(in_features=100, out_features=10, bias=True)
)

In [24]:
# Sau khi tiến hành share weight, ta cần tính toán lại accuracy
accuracy = test(model)

>>> Test set: Average loss: 0.0688, Accuracy: 9802/10000 (98.02%). Total time = 0.16124367713928223
